In [1]:
%%bash
cd /core
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

cd /nvtabular
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

cd /models
git config remote.origin.fetch "+refs/heads/*:refs/remotes/origin/*" && git fetch && git checkout main
pip install . --no-deps

From https://github.com/NVIDIA-Merlin/core
   9d9b5c6a..c5c9bc25 release-23.04       -> origin/release-23.04
 * [new branch]      feature/merlin-array-dispatch -> origin/feature/merlin-array-dispatch
 * [new branch]      fix-repartition     -> origin/fix-repartition
 * [new branch]      fix-with-properties -> origin/fix-with-properties
 * [new branch]      gh-pages            -> origin/gh-pages
 * [new branch]      laiacano/docs-on-pr -> origin/laiacano/docs-on-pr
 * [new branch]      main                -> origin/main
 * [new branch]      release-22.10       -> origin/release-22.10
 * [new branch]      release-22.11       -> origin/release-22.11
 * [new branch]      release-22.12       -> origin/release-22.12
 * [new branch]      release-23.02       -> origin/release-23.02
 * [new branch]      revert-163-refactor/dictarray-columns -> origin/revert-163-refactor/dictarray-columns
 * [new branch]      stable              -> origin/stable
 * [new branch]      tags-intersection   -> origin

Branch 'main' set up to track remote branch 'main' from 'origin'.
Processing /core
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for merlin-core: filename=merlin_core-23.5.dev0+28.ge1eaf269-py3-none-any.whl size=164561 sha256=700362e2fca0348cfdc2bd7dfb188985b727984a4844f5e604b89d110c8f35dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-xnol33be/wheels/8f/da/8c/c779661788874afaa32fd10abeac6016635956e3bad9940584
Successfully built merlin-core
  Attempting uninstall: merlin-core
    Found existing installation: merlin-core 23.4.0
    Uninstalling merlin-core-23.4.0:
      Successfully uninstalled merlin-core-23.4.0


From https://github.com/NVIDIA-Merlin/NVTabular
   f8f484e5..90489194 release-23.04           -> origin/release-23.04
 * [new branch]      1077-implement          -> origin/1077-implement
 * [new branch]      21.09/column-tagging    -> origin/21.09/column-tagging
 * [new branch]      21.09/dataset-collection -> origin/21.09/dataset-collection
 * [new branch]      21.09/operator-block    -> origin/21.09/operator-block
 * [new branch]      21.09/schema            -> origin/21.09/schema
 * [new branch]      add_sum_to_supported_aggregations -> origin/add_sum_to_supported_aggregations
 * [new branch]      aiobotocore_v2          -> origin/aiobotocore_v2
 * [new branch]      alexanderronquillo-patch-1 -> origin/alexanderronquillo-patch-1
 * [new branch]      atomize_adding_of_tags  -> origin/atomize_adding_of_tags
 * [new branch]      automate_pypi           -> origin/automate_pypi
 * [new branch]      bench-pynvml-fix        -> origin/bench-pynvml-fix
 * [new branch]      branch-0.6       

Branch 'main' set up to track remote branch 'main' from 'origin'.
Processing /nvtabular
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for nvtabular: filename=nvtabular-23.5.dev0+11.g7557ff03-cp38-cp38-linux_x86_64.whl size=261194 sha256=474771e2b56bf4622084d4117f357d1f51cb382cf8dcedbef5f155c9eabb408a
  Stored in directory: /tmp/pip-ephem-wheel-cache-h7yuc48r/wheels/df/bf/c2/9cc2a62fe6da42038c26a9c0c4e25f9767093528b102fa30a2
Successfully built nvtabular
  Attempting uninstall: nvtabular
    Found existing installation: nvtabular 23.4.0
    Uninstalling nvtabular-23.4.0:
      Successfully uninstalled nvtabular-23.4.0


From https://github.com/NVIDIA-Merlin/Models
   a44eced6..56c7d6a4 release-23.04           -> origin/release-23.04
 * [new branch]      Mai                     -> origin/Mai
 * [new branch]      add_category_encoding_test -> origin/add_category_encoding_test
 * [new branch]      add_lightfm_and_explicit_training_example -> origin/add_lightfm_and_explicit_training_example
 * [new branch]      add_logo_tracking_to_07 -> origin/add_logo_tracking_to_07
 * [new branch]      add_notebooks_test      -> origin/add_notebooks_test
 * [new branch]      add_sigir_dataset       -> origin/add_sigir_dataset
 * [new branch]      add_tf_docstrings_rny   -> origin/add_tf_docstrings_rny
 * [new branch]      advanced_example        -> origin/advanced_example
 * [new branch]      asvdb_metric_tracking   -> origin/asvdb_metric_tracking
 * [new branch]      batched-dataset/schema  -> origin/batched-dataset/schema
 * [new branch]      benchmark-session-based -> origin/benchmark-session-based
 * [new branch]  

 * [new branch]      retrieval_fixes         -> origin/retrieval_fixes
 * [new branch]      retrieval_fixes_2       -> origin/retrieval_fixes_2
 * [new branch]      retrieval_integration_tests -> origin/retrieval_integration_tests
 * [new branch]      revert-813-laiacano/tox-and-tmpdir -> origin/revert-813-laiacano/tox-and-tmpdir
 * [new branch]      romeyn/block-api        -> origin/romeyn/block-api
 * [new branch]      romeyn/block-cleanup    -> origin/romeyn/block-cleanup
 * [new branch]      romeyn/inputs           -> origin/romeyn/inputs
 * [new branch]      sampling                -> origin/sampling
 * [new branch]      select-by-tag           -> origin/select-by-tag
 * [new branch]      session-based/contrastive -> origin/session-based/contrastive
 * [new branch]      stable                  -> origin/stable
 * [new branch]      t4rec_use_case          -> origin/t4rec_use_case
 * [new branch]      tf/add-bokeh-to-dev     -> origin/tf/add-bokeh-to-dev
 * [new branch]      tf/base

 * [new branch]      validation_data_fix     -> origin/validation_data_fix
 * [new branch]      validation_data_fix2    -> origin/validation_data_fix2
 * [new branch]      wide_deep_example_test  -> origin/wide_deep_example_test
 * [new branch]      wideanddeep_example     -> origin/wideanddeep_example
 * [new branch]      xgboost/predict-without-target -> origin/xgboost/predict-without-target
 * [new branch]      youtube_dnn_retrieval   -> origin/youtube_dnn_retrieval
 * [new branch]      youtubednn_improv       -> origin/youtubednn_improv
 * [new branch]      youtubednn_logq         -> origin/youtubednn_logq
 * [new tag]         v0.10.0                 -> v0.10.0
 * [new tag]         v0.11.0                 -> v0.11.0
 * [new tag]         v0.9.0                  -> v0.9.0
 * [new tag]         v23.02.00               -> v23.02.00
 * [new tag]           v0.1.0                  -> v0.1.0
 * [new tag]           v0.2.0                  -> v0.2.0
 * [new tag]           v0.3.0              

Branch 'main' set up to track remote branch 'main' from 'origin'.
Processing /models
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for merlin-models: filename=merlin_models-23.5.dev0+17.g8efbd361-py3-none-any.whl size=357398 sha256=e4aa301d799d1832fc052f1fd69fe50634d244ec320cd9de0256679d2eed1a12
  Stored in directory: /tmp/pip-ephem-wheel-cache-fbcn70jg/wheels/4d/e8/98/0493db55fff90dc9af123f55a9455b96f7f8166c912a02c8a6
Successfully built merlin-models
  Attempting uninstall: merlin-models
    Found existing installation: merlin-models 23.4.0
    Uninstalling merlin-models-23.4.0:
      Successfully uninstalled merlin-models-23.4.0


In [2]:
import merlin.models.tf as mm
from merlin.schema.tags import Tags
import tensorflow as tf
from merlin.models.tf.blocks import *
from merlin.datasets.synthetic import generate_data
import pandas as pd

import numpy as np

2023-05-22 00:50:15.754273: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")


2023-05-22 00:50:18.196650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 00:50:18.197037: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 00:50:18.197163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-05-22 00:50:20.188904: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-22 00:50:20.189860: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 00:50:20.190036: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-22 00:50:20.190160: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must 

In [3]:
import cudf

In [4]:
ls /workspace/sigir_dataset/train

browsing_train.csv      search_train.csv
browsing_train.parquet  sku_to_content.csv


If we attempt reading in the data, we unfortunately get a `cudf` error.

In [5]:
cudf.read_csv('/workspace/sigir_dataset/train/browsing_train.csv')

RuntimeError: cuDF failure at: /opt/rapids/src/cudf/cpp/include/cudf/strings/detail/strings_column_factories.cuh:88: total size of strings is too large for cudf column

`cudf` can only read in datasets that have fewer than 2**32 characters. Here there is so much data and the strings are so long we cross this limit.

We could work with only partial data. But we would still have the problem of our id column consisting of strings. This is something that `datasets` is unable to work with for synthetic data generation.

Let's instead "categorify" the string columns of interest. We will have to do this nonetheless if we will want to work with the full dataset in `cudf`.

In [6]:
train = pd.read_csv('/workspace/sigir_dataset/train/browsing_train.csv')

In [7]:
train.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885210881,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
1,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,61ef3869355b78e11011f39fc7ac8f8dfb209b3442a9d5...,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
2,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885213307,4ed279f4f0deab6dfc80f4f7bf49d527fd894fa478a9ce...
3,20c458b802f6ea9374783bfc528b19421be977a6769785...,event_product,detail,d5157f8bc52965390fa21ad5842a8502bc3eb8b0930f3f...,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...
4,20c458b802f6ea9374783bfc528b19421be977a6769785...,pageview,NaN,NaN,1550885215484,7e4527ac6a32deed4f4f06bb7c49b907b7ca371e59d57d...


In [8]:
train.shape[0] / 1_000_000

36.079307

36+ million records!

In [9]:
len(train.iloc[0]['session_id_hash'])

64

In [10]:
2**32 / (train.shape[0] * 64)

1.860037500165954

We cannot accommodate even two hash columns!

In [11]:
train['session_id_hash'] = train['session_id_hash'].astype('category')
categories_index = train['session_id_hash'].cat.categories
train['session_id_hash'] = train['session_id_hash'].cat.codes

train['product_sku_hash'] = train['product_sku_hash'].astype('category')
categories_index = train['product_sku_hash'].cat.categories
hash_to_cat_idx = {h: i for i, h in enumerate(categories_index)}
train['product_sku_hash'] = train['product_sku_hash'].cat.codes

train['hashed_url'] = train['hashed_url'].astype('category')
categories_index = train['hashed_url'].cat.categories
train['hashed_url'] = train['hashed_url'].cat.codes

train.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,631811,event_product,detail,47936,1550885210881,241663
1,631811,event_product,detail,22091,1550885213307,151100
2,631811,pageview,NaN,-1,1550885213307,151100
3,631811,event_product,detail,47936,1550885215484,241663
4,631811,pageview,NaN,-1,1550885215484,241663


This is looking good and we are able to output and read in the file using `cudf`.

In [12]:
train.to_parquet('/workspace/sigir_dataset/train/browsing_train.parquet')
cudf.read_parquet('/workspace/sigir_dataset/train/browsing_train.parquet').head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,631811,event_product,detail,47936,1550885210881,241663
1,631811,event_product,detail,22091,1550885213307,151100
2,631811,pageview,<NA>,-1,1550885213307,151100
3,631811,event_product,detail,47936,1550885215484,241663
4,631811,pageview,<NA>,-1,1550885215484,241663


For training on the full dataset, we would need to apply similar processing to the sku information (using the hash to index mapping we collected above).

In [13]:
ls /workspace/sigir_dataset/train

browsing_train.csv      search_train.csv
browsing_train.parquet  sku_to_content.csv


In [14]:
skus = pd.read_csv('/workspace/sigir_dataset/train/sku_to_content.csv')

skus['product_sku_hash'] = skus['product_sku_hash'].map(hash_to_cat_idx).fillna(0).astype('int')

This will allow us to link sessions data to pretrained embeddings.

Let's now generate synthetic data.

In [26]:
train = pd.read_csv('/workspace/sigir_dataset/train/browsing_train.csv').head(10_000)

train['session_id_hash'] = train['session_id_hash'].astype('category')
categories_index = train['session_id_hash'].cat.categories
train['session_id_hash'] = train['session_id_hash'].cat.codes

train['product_sku_hash'] = train['product_sku_hash'].astype('category')
categories_index = train['product_sku_hash'].cat.categories
hash_to_cat_idx = {h: i for i, h in enumerate(categories_index)}
train['product_sku_hash'] = train['product_sku_hash'].cat.codes

train['hashed_url'] = train['hashed_url'].astype('category')
categories_index = train['hashed_url'].cat.categories
train['hashed_url'] = train['hashed_url'].cat.codes

train.head()

,session_id_hash,event_type,product_action,product_sku_hash,server_timestamp_epoch_ms,hashed_url
0,194,event_product,detail,1257,1550885210881,1320
1,194,event_product,detail,593,1550885213307,832
2,194,pageview,NaN,-1,1550885213307,832
3,194,event_product,detail,1257,1550885215484,1320
4,194,pageview,NaN,-1,1550885215484,1320


In [53]:
train.session_id_hash.nunique()

1363

In [85]:
from nvtabular import *
import nvtabular as nvt

In [94]:
out = ['session_id_hash'] >> nvt.ops.Categorify(max_size=1000) >> nvt.ops.TagAsItemID()
out += ['event_type', 'product_action', 'product_sku_hash', 'hashed_url'] >> nvt.ops.Categorify(max_size=1000)
out += ['server_timestamp_epoch_ms'] >> nvt.ops.NormalizeMinMax()

In [95]:
wf = Workflow(out)

In [96]:
ds = Dataset(train)
ds_out = wf.fit_transform(ds)

In [97]:
ds_out.to_parquet('ds_out')

In [98]:
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

pb = TensorflowMetadata.from_proto_text_file(path='ds_out')
pb.to_json_file(path='ds_out')

!python3 -m json.tool ds_out/schema.json

{
    "feature": [
        {
            "name": "session_id_hash",
            "type": "INT",
            "intDomain": {
                "name": "session_id_hash",
                "max": "999",
                "isCategorical": true
            },
            "annotation": {
                "tag": [
                    "id",
                    "categorical",
                    "item"
                ],
                "extraMetadata": [
                    {
                        "num_buckets": null,
                        "freq_threshold": 0.0,
                        "max_size": 1000.0,
                        "start_index": 0.0,
                        "cat_path": ".//categories/unique.session_id_hash.parquet",
                        "embedding_sizes": {
                            "cardinality": 1000.0,
                            "dimension": 77.0
                        },
                        "_dims": [
                            [
                                0.0,


In [99]:
from merlin.datasets.synthetic import generate_data

ds_done = generate_data('ds_out/schema.json', 1000)

In [101]:
ds_done.head()

,session_id_hash,event_type,product_action,product_sku_hash,hashed_url,server_timestamp_epoch_ms
0,25,1,2,0,612,0.993113
1,179,0,3,665,514,0.017887
2,173,0,4,336,994,0.507542
3,4,1,4,434,555,0.178694
4,10,1,3,123,412,0.630324


In [106]:
ds_done.compute().shape

(1000, 6)

In [107]:
ds_done.compute()['session_id_hash'].value_counts()

11     43
9      43
6      35
4      34
8      31
       ..
162     1
160     1
175     1
76      1
248     1
Name: session_id_hash, Length: 127, dtype: int32

Ok, so we got the schema 🙂 But the question is what to do with the sku data.

In [110]:
skus = pd.read_csv('/workspace/sigir_dataset/train/sku_to_content.csv')
skus.head()

,product_sku_hash,description_vector,category_hash,image_vector,price_bucket
0,26ce7b47f4c46e4087e83e54d2f7ddc7ea57862fed2e2a...,NaN,NaN,NaN,NaN
1,6383992be772b204a9ab75f86c86f5583d1bdd1222952d...,NaN,NaN,NaN,NaN
2,a2c3e2430c6ef9770b903ad08fa067a6b2b9db28f06e1b...,"[0.27629122138023376, -0.15763211250305176, 0....",06fa312761d4b39e2f649781514ac69a4c1505c221fc46...,"[340.3592564184389, -220.19025864725685, 154.0...",7.0
3,1028ef615e425c328e7b95010dfb1fb93cf63749a1bc80...,"[0.4058118760585785, -0.03595402091741562, 0.2...",115a6a7017ee55752b8487c77dfde92b0d501d10a2e69c...,"[180.3463662921092, 222.702322343354, -8.88703...",8.0
4,9870c682d0d52d635501249da0eeaa118fad430b695ea1...,"[-0.3206155300140381, 0.01991105079650879, 0.0...",0665a81d19c89281cc00e7f7d779ded2ed42c933838602...,"[-114.81079301576219, 84.55770104232334, 85.51...",2.0


I would suggest that in the example we create two branches here, if someone is using the full dataset, they can map the sku information to the product information using the dict we created above like so:

In [111]:
skus['product_sku_hash'] = skus['product_sku_hash'].map(hash_to_cat_idx).fillna(0).astype('int')

And for the people taking the synthetic data route we can generate the sku information (description vectors and potentially also image vectors) as a separate branch.

Once done, the two branches will merge and the rest of the example will be the same for both of the groups.